# Deep Dream Journey

This notebook allows you to travel through Google's Deep Dream worlds. You can check out Google's app [here](http://deepdreamgenerator.com/) if you don't have an idea of what it is, and [here](https://github.com/google/deepdream) is their notebook with the sources of the project.

Google's notebook describes the technology behind their app and how it works. This notebook allows you to travel through the worlds generated by Google!

Specifically, you'll be able to generate frames of "deep" dreams (when you feed the output of the network back as an input) and create a video from them. You'll also be able to specify where to zoom the picture, so that you select the direction of where to "go" in the journey.

In general, the notebook provides a nice setup, bringing your image and Google's technology in one context.

The github [repository](https://github.com/anatoliykmetyuk/deepdream-journey) contains the latest version of this notebook, make sure to check it out.

The project is based on Google's Deep Dream notebook code, it is incapsulated in the [deepdream.py](./deepdream.py) module.

## Setup

Import the libraries to be used. The dependencies of this notebook are same as of the Google's deepdream one. You may need to do some heavy setup before running this notebook.

The recommended way to install most of the dependencies is to install [Anaconda](https://docs.continuum.io/anaconda/pkg-docs). However, it will not install [PIL](http://www.pythonware.com/products/pil/) and [Caffe](http://caffe.berkeleyvision.org/installation.html) for you - you will need to compile them from sources.

Caffe is a work in progress, the hardest part is to get it working. Make sure you follow the instructions on their website and have all the dependencies. If you get a segmentation fault 11 when importing caffe, refer to [this](https://github.com/BVLC/caffe/issues/2677#issuecomment-192470944) solution.

Some imports are from the modules from this notebook's root - `deepdream.py` and `deepfront.py`. The former contains Google's code from its deepdream notebook - credit for them. The latter one contains some helpers that are not part of the Google's notebook.

In [ ]:
%matplotlib inline

# imports and basic notebook setup
from cStringIO import StringIO
import numpy as np
import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from google.protobuf import text_format

import caffe
from deepdream  import deepdream, load_model, showarray
from dreamfront import load_img, dream_layers, dream_deep

import calendar
import time
import os

# Function to measure time
def time_sec():
    return calendar.timegm(time.gmtime())

# If your GPU supports CUDA and Caffe was built with CUDA support,
# uncomment the following to run Caffe operations on the GPU.
# caffe.set_mode_gpu()
# caffe.set_device(0) # select GPU device if multiple devices exist

Next, we need to load our neural network. You don't have it out of the box, you'll have to download it from [here](https://github.com/BVLC/caffe/tree/master/models/bvlc_googlenet). Place it under your caffe/models/bvlc_googlenet folder. Then specify the path to your caffe clone under `caffe_path` variable below.

In [ ]:
caffe_path = '/Users/anatolii/Projects/clones/caffe'  # substitute your path here
model_path = os.path.join(caffe_path, 'models/bvlc_googlenet/')
model, net = load_model(model_path)

Load an image you will be working with.

In [ ]:
base = '.' # Directory where the image is located
name = 'sky1024px.jpg' # Name of the image to be opened
img = load_img(name, base, scale=0.7) # If the image takes too long to be processed, set `scale` between 0 and 1
print(img.shape)
showarray(img)

## Deepdream function

Let's check whether everything is working and measure how long does `deepdream` function with the standard parameters take.

In [ ]:
t1 = time_sec()
dream = deepdream(net, img)
time_per_dream = time_sec() - t1

print('Dream took %i seconds' % time_per_dream)
showarray(dream)

## Explore how all the layers react to deepdream

We will run `deepdream` function on all the layers available. You can select a layer with `end` parameter to the `deepdream` function. `dream_layers` function does this for you; you can check it out in the [dreamfront.py](./dreamfront.py) file.

Run the following code to see all the layers available.

In [ ]:
net.blobs.keys()

Prepare the directory where you want the output to be generated.

In [ ]:
all_layers_dir='all_layers'
!mkdir {all_layers_dir}
!rm {all_layers_dir}/*

Run `deepdream` on all the layers. `iter_n` and `octave_n` will be propagated to the `deepdream` function. `octave_n` is intentionally reduced (originally 4) to speed up things. After running this code, check out your `all_layers_dir` directory for the output.

In [ ]:
dream_layers(net, img, 'all_layers', iter_n=10, octave_n=3)

## Go deeper!

Here, we feed the outout of `deepdream` back to the network as an input, hence increasing the effect. We save the result of each iteration as an image to the directory.

First, prepare this directory.

In [ ]:
frames_dir='frames'
!mkdir {frames_dir}
#!rm {frames_dir}/* # Uncomment this if you want the directory to be cleared

Generate 40 frames, write them to `frames_dir`. You can do more, by changing `frames_n` variable. Also, `iter_n` and `octave_n` are reduced compared to the default ones (originally - 10 and 4 correspondently). Not only this is done for better speed, but also for smoother change between the frames. There are many other parameters to tune, see [deepdream.py](./deepdream.py) from the root of the project for all the parameters you can supply to `deepdream`.

In [ ]:
dream_deep(net, img, frames_dir, frames_n=40, iter_n=3, octave_n=3)

## Generate more frames

Just in case your journey into the `deep_dream` was interrupted (or you decided you want more), you can resume from when you ended.

First, we search for the last frame generated and output it along with its id.

In [ ]:
frames_paths = [p for p in os.listdir(frames_dir) if os.path.isfile(os.path.join(frames_dir, p))]
last_frame_name = frames_paths[-1]
last_frame = load_img(last_frame_name, frames_dir)
last_frame_id = int(os.path.splitext(last_frame_name)[0]) # ID of the last frame in the frames directory

print(last_frame_id)
showarray(last_frame)

Now we continue the generation from the point we ended it.

You have probably noticed that the image zooms with each iteration. By default, it zooms on the center. You can change that by passing `dh` and `dw` parameters to `deep_dream`. They specify the coordinate to zoom in, `dh` is a vertical coordinate (`h` for height) and `dw` - the horizontal one (`w` for width). They can take values from 0 to 1, (0, 0) being the upper left corner and (1, 1) - the lower right one. The default parameters are (0.5, 0.5) - for the center.

Since zoom resembles joyrney very much, this technique allows you to choose where to travel. Let's go to the right now, also for 40 frames.

In [ ]:
dream_deep(net, last_frame, frames_dir, start_idx=last_frame_id+1, frames_n=40, iter_n=3, octave_n=3, dh=0.5, dw=0.75)

## Make a video

All these frames from the previous task look very much like they can be the frames of a video. So let's make a video out of them.

First, let's check whether you have `ffmpeg` on your system.

In [ ]:
!which ffmpeg

If it produced no output, you don't have it. Install it using your system's package manager. For example, on OS X you can use [homebrew](http://brew.sh/): `brew install ffmpeg`.

In [ ]:
fps=8 # Frames per second
filename='movie.mp4' # Where to save the video

!ffmpeg -f image2 -r {fps} -i {frames_dir}/%06d.jpg -vcodec mpeg4 -y {filename}

As you can see, the depth of the world is lost to the end of the journey, so you won't probably be able to travel far like that. Though experimentation never hurts - maybe you'll be the one to find a way! :)